# DAGs - a generalization of the alignment problem
##Specification
- Name: problem15 in Rosalind
- Name your notebook as: DAG.ipynb
- options: none
- input: filename passed as first parameter to main
- output: a text file. ( using print ( .... file=someFileObject) is a handy way to do this after you have opened someFileObject as a text file). I find it handy to name these files by creating a string by concatenating the string named infile with ".out" ... rosalind5.txt.out ( for example).
-Rosalind Problem Name: Find the longest path in a DAG

As always, include a Inspection Intro Markdown that describes your specific algorithm at the beginning of the notebook, and another Inspection Results markdown at the end of the notebook that documents: your inspection team, the findings of the team, and your resolution of those findings.

Please submit your notebook, an example of one of the Rosalind files that you ran and passed, and the output that your program generated as a text file.

## Description
As a prelude to an upcoming assignment in profile alignments, please consider the related problem of generalized weighted-DAGs (directed acyclic graph). Rosalind has a very nice example that includes weighted edges connecting nodes. As applied to a simple pairwise alignment, nodes represent a specific assignment of a character pairing between a character in one of the sequences to either an initial insert/delete (indel), an extension of that indel, or a match/mismatch. 

Unlike classical pairwise alignment, the specific order to consider the resolution of scores in the graph is established using the topology of the graph ( topological ordering). This ordering will avoid recursion in developing the final score. This is another way of using Dynamic programming in a more generalized way. ( pp247-52 in Compeau and the detour on pp287-88).

## Hints
1) scoring of nodes that have no inputs (or possibly incomplete inputs) must be done carefully. Choose these wisely. In a pairwise alignment, these nodes would be considered boundary nodes.

2) The problem is quite clear on the starting/ending node to consider. Make sure to evaluate these properly.

3) each node can have many inputs or none. In the generalized DAG, there is no limit on the number of inputs. Consider a data structure here that allows you to represent the incoming edge along with a score that is computed by adding the weight of that incoming edge to the score of its preceding node.

4) As you write these classes, consider that these may be usable in the future.

5) sets and dictionaries require keys that are immutable. Those keys can be integers, or floats, or tuples. Think carefully about using tuples as containers-that-can-be-keys.

6) python has a container called a namedtuple. These are available in the container module. They are quite handy.

7) python has two functions called any() and all(); any() evaluates to true if any of its iterable arguments are True; all() evaluates to True if all of its iterable arguments are true. None, for example, evaluates to False. 

8) consider implementing a helper method called _argMax_. If you happen to have a dictionary of values, it returns the key whose paired-value is maximal among all values in the dictionary.

9) Carefully consider the implementations that are provided in the text. Specifically, the topology ordering pseudocode removes edges from the constructed DAG. This is a bit annoying, considering that we need that DAG for scoring and backtracking. What is the purpose that removing an edge is attempting to perform? Look over the Wikipedia article on "Dataflow" for an idea as to how you might avoid destroying your graph.

10) My implementation has specific objects that are nodes, edges, and the DAG itself. It is implemented in about 120 lines including whitespace.

11) Rosalind will provide test cases where the proscribed beginning and ending node have inputs/outputs that may induce cycles. In these problems, remember that we are considering the DAG that is described between (inclusively) the start and end node so the inputs to start don't matter; the outputs from end don't matter either.

 

Here is a template to use.

## Inspection Intro

In [8]:
#ash
import sys

class node:
    """ Represents a node in a Directed Acyclic Graph (DAG). """

    def __init__(self, label):
        """
        Initialize a new node with a label.
        
        Parameters:
        label (str): The label of the node.
        """
        # List of parent nodes and the weights of their edges.
        self.label = label
        self.parent_nodes = []  
        # List of target nodes and the weights of their edges.
        self.target_nodes = [] 
        # Boolean flag to mark if the node has been visited. 
        self.visited = False    

class DAG:
    """ Represents a Directed Acyclic Graph (DAG) and operations to perform on it. """

    def __init__(self):
        """
        Initialize a new DAG.
        """
        # Dictionary to store nodes with their labels as keys.
        self.nodes_dict = {} 
        # Dictionary to store the longest distance from source to each node. 
        self.distance = {} 
         # Dictionary to store the previous node for the longest path.   
        self.backtrack = {}  
    def add_node(self, label):
        """
        Add a new node to the DAG, or return it if it already exists.

        Parameters:
        label (str): The label of the node to add or retrieve.

        Returns:
        node: The new or existing node.
        """
        if label in self.nodes_dict:
            return self.nodes_dict[label]

        new_node = node(label)
        self.nodes_dict[label] = new_node
        return new_node

    def contruct_dag(self, adj_list_text):
        """
        Construct the DAG using an adjacency list text representation.

        Parameters:
        adj_list_text (list of str): The adjacency list where each element represents an edge.
        """
        for line in adj_list_text:
            nodeA, tmp = line.split("->")
            nodeB, weight = tmp.split(":")
            weight = int(weight)

            from_node = self.add_node(nodeA)
            to_node = self.add_node(nodeB)

            from_node.target_nodes.append((to_node, weight))
            to_node.parent_nodes.append((from_node, weight))

    def topo_sort_u(self, node, stack):
        """
        Utility function for topological sort.

        Parameters:
        node (node): The current node to process.
        stack (list of str): The stack to keep track of the sorted nodes.
        """
        node.visited = True
        for node2,_ in node.target_nodes:
            if not node2.visited:
                self.topo_sort_u(node2, stack)
        stack.insert(0, node.label)
        
    def topo_sort(self):
        """
        Perform topological sort on the DAG.

        Returns:
        list of str: The list of node labels in topologically sorted order.
        """
        stack = []
        for node in self.nodes_dict.values():
            if not node.visited:
                self.topo_sort_u(node, stack)
        return stack

    def long_path(self, source, sink):
        """
        Find the longest path in the DAG from a source node to a sink node.

        Parameters:
        source (str): The label of the source node.
        sink (str): The label of the sink node.

        Returns:
        tuple: A tuple containing the longest distance and the path as a list of node labels.
        """
        for label in self.nodes_dict:
            self.distance[label] = -float("Inf")

        self.distance[source] = 0
        self.backtrack[source] = None

        top_order = self.topo_sort()
        for label in top_order:
            current_node = self.nodes_dict[label]
            for v, weight in current_node.target_nodes:
                if self.distance[v.label] < self.distance[label] + weight:
                    self.distance[v.label] = self.distance[label] + weight
                    self.backtrack[v.label] = label

        path = [sink]
        curr = self.backtrack[sink]
        while curr != source:
            path = [curr] + path
            curr = self.backtrack[curr]
        path = [source] + path
        return self.distance[sink], path


if __name__ == "__main__":
    # Check if the input file name is provided.
    if len(sys.argv) < 2:
        print("Usage: python script.py filename")
        sys.exit(1)

    # The input file name.
    infile = sys.argv[1]

    # Open the input file to read the adjacency list text.
    with open(infile, 'r') as file:
        input_data = file.read().splitlines()

    # The source node label.
    source = input_data[0]
    # The sink node label.
    sink = input_data[1]
    # The adjacency list text.
    adj_list_text = input_data[2:]

    # Create and construct the DAG.
    graph = DAG()
    graph.contruct_dag(adj_list_text)

    # Find the longest path.
    longest_dist, long_path = graph.long_path(source, sink)

    # Output file name.
    outfile = infile + ".out" # changed for jupyter idk if this helps it actually run

    # Open the output file to write the results.
    with open(outfile, 'w') as file:
        print(longest_dist, file=file)
        print("->".join(long_path), file=file)

    print(f"Results written to {outfile}")
    # # Read the input from standard input.
    # input = sys.stdin.read().splitlines()
    # # The source node label.
    # source = input[0] 
    # # The sink node label. 
    # sink = input[1]    
    # adj_list_text = input[2:]  
    # # The adjacency list text.
    # graph = DAG()
    # graph.contruct_dag(adj_list_text)
    # longest_dist, long_path = graph.long_path(source, sink)
    # print(longest_dist)
    # print("->".join(long_path))


FileNotFoundError: [Errno 2] No such file or directory: '-f'

Pre-inspection:

Class node:
Represents a node in the graph.
Each node has a label, lists to keep track of its parent and target nodes (with associated weights), and a visited flag for graph traversal algorithms.
Class DAG:
Represents the entire graph.
Contains a dictionary to store nodes (nodes_dict), a dictionary to store the longest distances from the source to each node (distance), and a dictionary to store the previous node for the longest path (backtrack).
Methods:
add_node: Adds a new node to the graph or retrieves it if it already exists.
construct_dag: Constructs the graph using an adjacency list text representation, where each list element represents an edge in the format "nodeA->nodeB:weight".
topo_sort_u: A utility function for topological sort, which marks nodes as visited and inserts them into a stack in topological order.
topo_sort: Performs a topological sort on the graph and returns the sorted nodes' labels.
long_path: Finds the longest path from a specified source node to a sink node using the topologically sorted order of the nodes.

## Inspection Results

The issue my team had with my code is that we couldnt get it to work in jupyter notebooks. We werent able to fix the issue, but when the code is run in terminal it come up with the right input. We decdied to just ignore this issue, since my code does work.